In [1]:
from __future__ import print_function
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from collections import Counter

In [2]:
import os
for dirname, _, filenames in os.walk('./data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


./data/gender_submission.csv
./data/train.csv
./data/test.csv


## Load train and test file with pandas

In [3]:
train = pd.read_csv("./data/train.csv")
test = pd.read_csv("./data/test.csv")

In [4]:
full = [train, test]

## Visualizing data

In [5]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


### Dataset visualization and statistics

In [6]:
#Let's see the amount of survived for each class. That can be done easily with pandas and its groupby method
PclassRatio = train[['Pclass', 'Survived']].groupby(['Pclass']).mean()
print(PclassRatio)

        Survived
Pclass          
1       0.629630
2       0.472826
3       0.242363


In [7]:
SexRatio = train[['Sex', 'Survived']].groupby(['Sex']).mean()
print(SexRatio)

        Survived
Sex             
female  0.742038
male    0.188908


In [15]:
for dataset in full:
    #Start creating new features for both dataset train and test
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

FamilySizeRatio = train[['FamilySize', 'Survived']].groupby(['FamilySize']).mean()
print(FamilySizeRatio)
# FamilySizeRatio.plot.barh()

            Survived
FamilySize          
1           0.303538
2           0.552795
3           0.578431
4           0.724138
5           0.200000
6           0.136364
7           0.333333
8           0.000000
11          0.000000


In [35]:
for dataset in full:
    dataset['Fare'] = dataset['Fare'].fillna(train['Fare'].median())
train['QuartileFare'] = pd.qcut(train['Fare'],4)
train['DecilesFare'] = pd.qcut(train['Fare'], 10)

QuartileFareRatio = train[['QuartileFare', 'Survived']].groupby(['QuartileFare']).mean()
DecilesFareRatio = train[['DecilesFare', 'Survived']].groupby(['DecilesFare']).mean()
print('Quartiles Fare \n {} \n'.format(QuartileFareRatio))
print('Deciles Fare \n {}\n'.format(DecilesFareRatio))


Quartiles Fare 
                  Survived
QuartileFare             
(-0.001, 7.91]   0.197309
(7.91, 14.454]   0.303571
(14.454, 31.0]   0.454955
(31.0, 512.329]  0.581081 

Deciles Fare 
                    Survived
DecilesFare                
(-0.001, 7.55]     0.141304
(7.55, 7.854]      0.298851
(7.854, 8.05]      0.179245
(8.05, 10.5]       0.230769
(10.5, 14.454]     0.428571
(14.454, 21.679]   0.420455
(21.679, 27.0]     0.516854
(27.0, 39.688]     0.373626
(39.688, 77.958]   0.528090
(77.958, 512.329]  0.758621

